# 판다스 이용해 타이타닉 데이터

In [2]:
# titanic_train 

import pandas as pd
df_titanic = pd.read_csv('./datasets/titanic_train.csv')
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
df_titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
# loc를 활용하여 feature와 label만으로 이루어진 데이터프레임 만들기
# Survived, Sex, Age, Pclass

df_titanic = df_titanic.loc[:,['Survived', 'Sex', 'Age', 'Pclass']]
df_titanic

,Survived,Sex,Age,Pclass
0,0,male,22.0,3
1,1,female,38.0,1
2,1,female,26.0,3
3,1,female,35.0,1
4,0,male,35.0,3
...,...,...,...,...
886,0,male,27.0,2
887,1,female,19.0,1
888,0,female,NaN,3
889,1,male,26.0,1


In [6]:
# feature를 X로 저장
# label을 y로 저장

X = df_titanic.loc[:,['Sex', 'Age', 'Pclass']]
y = df_titanic['Survived']

# 결측치 처리

In [7]:
# 결측치를 ffill을 이용해서 처리

X['Age'].fillna(method='ffill', inplace=True)

# Label Encoding - Sex

In [8]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

labels = encoder.fit_transform(X['Sex'])

X['Sex'] = labels

In [9]:
X

,Sex,Age,Pclass
0,1,22.0,3
1,0,38.0,1
2,0,26.0,3
3,0,35.0,1
4,1,35.0,3
...,...,...,...
886,1,27.0,2
887,0,19.0,1
888,0,19.0,3
889,1,26.0,1


# feature Scaling - Age
## MinMaxScaler 이용


### 방법 1

In [10]:
# MinMaxScaler 이용해 Age 스케일링

from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 객체 생성
scaler = MinMaxScaler()

# MinMaxScaler로 데이터셋 변환, fit()과 transfom() 호출
scaler.fit(X[['Age']])
age_scaled = scaler.transform(X[['Age']])
X['Age']=age_scaled

### 방법 2

In [11]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()
age = np.array(X['Age']).reshape(-1, 1)
age = scaler.fit_transform(age)
X['Age'] = age

In [12]:
X

,Sex,Age,Pclass
0,1,0.271174,3
1,0,0.472229,1
2,0,0.321438,3
3,0,0.434531,1
4,1,0.434531,3
...,...,...,...
886,1,0.334004,2
887,0,0.233476,1
888,0,0.233476,3
889,1,0.321438,1


# DecisionTreeClassifier와 cross_val_score를 활용한 학습

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate

dt_clf = DecisionTreeClassifier(random_state=156)

scores = cross_val_score(dt_clf, X, y, scoring='accuracy', cv = 5)

In [14]:
scores.mean()

0.7889962965287804

# 예측 및 평가

## test 데이터 불러오기

In [15]:
test_titanic = pd.read_csv('./datasets/titanic_test.csv')
test_titanic.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## test 데이터 전처리 (train데이터에 한 것 그대로)

In [16]:
# feature만 생성
X_test = test_titanic.loc[:,['Sex', 'Age', 'Pclass']]

# 결측값 채우기
X_test['Age'].fillna(method='ffill', inplace=True)

# Label Encoding - Sex
encoder = LabelEncoder()
labels = encoder.fit_transform(X_test['Sex'])
X_test['Sex'] = labels

# feature Scaling - Age
scaler = MinMaxScaler()
age = np.array(X_test['Age']).reshape(-1, 1)
age = scaler.fit_transform(age)
X_test['Age'] = age

In [17]:
X_test

,Sex,Age,Pclass
0,1,0.452723,3
1,0,0.617566,3
2,1,0.815377,2
3,1,0.353818,3
4,0,0.287881,3
...,...,...,...
413,1,0.367005,3
414,0,0.512066,1
415,1,0.505473,3
416,1,0.505473,3


## 예측

In [19]:
dt_clf.fit(X, y)
pred = dt_clf.predict(X_test)
pred

array([0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,

### GridSearchCV 활용예측

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

dt_clf = DecisionTreeClassifier(random_state=121)
parameters = {'max_depth' : [1,2,3], 'min_samples_split' : [2,3]}

grid_dtree = GridSearchCV(dt_clf, param_grid=parameters, cv=5, refit=True)

# 붓꽃 Train 데이터로 param_gid의 하이퍼 파라미터들을 순차적으로 학습/평가
grid_dtree.fit(X, y)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=121),
             param_grid={'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]})

In [22]:
best_dt_clf = grid_dtree.best_estimator_

In [23]:
best_dt_clf.predict(X_test)

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,